# ACINHIBITOR RUNS

Running the models in python.

In [1]:
from __future__ import print_function

from time import time
import os
import logging
import scipy
import scipy.sparse as sp
import numpy as np
import pandas as pd


import gensim
import re
from gensim import models, corpora, similarities
from collections import defaultdict
import nltk
from nltk.corpus import stopwords


from sklearn import metrics, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from scipy.stats.distributions import randint
from sklearn.cross_validation import StratifiedKFold, train_test_split, cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, Perceptron, PassiveAggressiveClassifier

#

print ("Required Libraries loaded.")

C:\Users\CR107\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Required Libraries loaded.


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### import file in cvs format

In [3]:
#load stored corpus from disk
dict_path = "C:\Users\CR107\Dropbox\PhD\Experiments\TREC_2004\LSI/dict/aceinhibitor.dict"
mm_path = "C:\Users\CR107\Dropbox\PhD\Experiments\TREC_2004\LSI/dict/aceinhibitor.mm"
if (os.path.exists(dict_path)):
    dictionary = corpora.Dictionary.load(dict_path)
    corpus = corpora.MmCorpus(mm_path)
    print("Needed files loaded from disk")
else:
    print("Please run the data_prep.ipynb file to generate data set")

print (corpus)

2017-01-16 12:44:22,167 : INFO : loading Dictionary object from C:\Users\CR107\Dropbox\PhD\Experiments\TREC_2004\LSI/dict/aceinhibitor.dict
2017-01-16 12:44:22,262 : INFO : loaded corpus index from C:\Users\CR107\Dropbox\PhD\Experiments\TREC_2004\LSI/dict/aceinhibitor.mm.index
2017-01-16 12:44:22,263 : INFO : initializing corpus reader from C:\Users\CR107\Dropbox\PhD\Experiments\TREC_2004\LSI/dict/aceinhibitor.mm
2017-01-16 12:44:22,323 : INFO : accepted corpus with 2498 documents, 10730 features, 259641 non-zero entries


Needed files loaded from disk
MmCorpus(2498 documents, 10730 features, 259641 non-zero entries)


#### Transform the corpus

In [44]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

corpus_tfidf = tfidf[corpus] ##apply the transformation to all corpus
#for doc in corpus_tfidf:
#    print(doc)


lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2000) # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

print ("Done.")

#lsi_model.save('/tmp/model.lsi') # same for tfidf, lda, ...
#lsi_model = models.LsiModel.load('/tmp/model.lsi')

2017-01-16 16:15:27,769 : INFO : collecting document frequencies
2017-01-16 16:15:27,770 : INFO : PROGRESS: processing document #0
2017-01-16 16:15:28,678 : INFO : calculating IDF weights for 2498 documents and 10729 features (259641 matrix non-zeros)
2017-01-16 16:15:28,686 : INFO : using serial LSI version on this node
2017-01-16 16:15:28,687 : INFO : updating model with new documents
2017-01-16 16:15:29,815 : INFO : preparing a new chunk of documents
2017-01-16 16:15:29,858 : INFO : using 100 extra samples and 2 power iterations
2017-01-16 16:15:29,858 : INFO : 1st phase: constructing (10730L, 2100L) action matrix
2017-01-16 16:15:30,591 : INFO : orthonormalizing (10730L, 2100L) action matrix
2017-01-16 16:15:40,273 : INFO : 2nd phase: running dense svd on (2100L, 2498L) matrix
2017-01-16 16:15:45,832 : INFO : computing the final decomposition
2017-01-16 16:15:45,834 : INFO : keeping 2000 factors (discarding 0.819% of energy spectrum)
2017-01-16 16:15:46,799 : INFO : processed docum

Done.


In [40]:
lsi_model.print_debug(num_topics=5, num_words=5)
lsi_model.print_topic(topicno=5, topn=5)
lsi_model.print_topics(num_topics=5, num_words=5)
#for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
#     print(doc)

2017-01-16 13:47:47,040 : INFO : computing word-topic salience for 5 topics
2017-01-16 13:47:47,210 : INFO : topic #0(8.850): patients(0.076), study(0.062), middle(0.027), treatment(0.072), aged(0.043), ..., 
2017-01-16 13:47:47,223 : INFO : topic #1(4.832): heart(-0.189), failure(-0.215), myocardial(-0.202), ventricular(-0.184), infarction(-0.161), ..., hg(0.189), mm(0.183)
2017-01-16 13:47:47,240 : INFO : topic #2(4.296): renal(-0.284), diabetic(-0.266), diabetes(-0.229), mellitus(-0.127), type(-0.194), ..., ventricular(0.202), left(0.173)
2017-01-16 13:47:47,256 : INFO : topic #3(4.086): losartan(0.480), receptor(0.169), angiotensin(0.162), angiotensinantagonists(0.081), ii(0.131), ..., left(-0.120), ventricular(-0.119)
2017-01-16 13:47:47,272 : INFO : topic #4(3.860): losartan(-0.277), plasma(-0.126), captopril(-0.163), measured(-0.038), enalapril(-0.134), ..., cardiovascular(0.168), risk(0.150)
2017-01-16 13:47:47,279 : INFO : topic #0(8.850): 0.140*"enalapril" + 0.122*"captopril"

[(0,
  u'0.140*"enalapril" + 0.122*"captopril" + 0.120*"mg" + 0.118*"failure" + 0.118*"heart"'),
 (1,
  u'0.228*"bp" + -0.215*"failure" + -0.202*"myocardial" + 0.189*"hg" + -0.189*"heart"'),
 (2,
  u'-0.284*"renal" + -0.266*"diabetic" + -0.229*"diabetes" + 0.202*"ventricular" + -0.194*"type"'),
 (3,
  u'0.480*"losartan" + 0.169*"receptor" + 0.164*"cough" + 0.162*"angiotensin" + 0.142*"failure"'),
 (4,
  u'-0.277*"losartan" + 0.168*"cardiovascular" + -0.163*"captopril" + 0.150*"risk" + -0.134*"enalapril"')]

In [42]:
lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=2000)
#lda_model.print_topics(5)

2017-01-16 16:09:59,809 : INFO : using symmetric alpha at 0.0005
2017-01-16 16:09:59,813 : INFO : using symmetric eta at 0.0005
2017-01-16 16:09:59,818 : INFO : using serial LDA version on this node
2017-01-16 16:13:42,131 : INFO : running online LDA training, 2000 topics, 1 passes over the supplied corpus of 2498 documents, updating model once every 2000 documents, evaluating perplexity every 2498 documents, iterating 50x with a convergence threshold of 0.001000
2017-01-16 16:13:42,134 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2017-01-16 16:13:43,006 : INFO : PROGRESS: pass 0, at document #2000/2498
2017-01-16 16:14:43,947 : INFO : merging changes from 2000 documents into a model of 2498 documents
2017-01-16 16:14:46,496 : INFO : topic #1869 (0.001): 0.001*pregnancy + 0.001*perinatal + 0.001*nephropathiesdiagnosisdrug + 0.001*adequate + 0.001*reproductive + 0.001*renal + 0.001*fetal + 0.001*albu

In [43]:
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus_lsi)

In [12]:
scipy_matrix = scipy_csc_matrix.T
scipy_matrix.shape

(2498, 500)

In [13]:
# Import the data and explore the first few rows
# Import the data and explore the first few rows

inhibitor  = pd.read_csv("C:\EPC_Data\TREC_BROKEN\No_Mh_Tag/aceinhibitor_no_mh.csv", sep=",")#, index_col = "PMID")
seed = 55
# shuffle dataset and split to train and test
x, scipy_matrix = shuffle(inhibitor, scipy_matrix, random_state = seed)
y_CV = x.Label

#split to test and train
X_train, X_test, y_train, y_test = train_test_split(scipy_matrix, x.Label, random_state = 2)

## Build models

### fit svm

In [18]:
##fit SVM
svm = SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
svm.fit(X_train, y_train)
svmpred = svm.predict(X_test)

print("Training set score: %f" % svm.score(X_train, y_train))
print("Test set score: %f" % svm.score(X_test, y_test))

# Get the confusion matrix for your classifier using metrics.confusion_matrix
mat = metrics.confusion_matrix(y_test, svmpred)    
print(mat)


print(metrics.classification_report(y_test, svmpred))
print("Overall Accuracy:", round(metrics.accuracy_score(y_test, svmpred),2))


# get number of support vectors for each class 
print("Number of support vectors for each class: ", svm.n_support_)

#Indices of support vectors
#print("Support vector indices: ", svm.support_)

Training set score: 1.000000
Test set score: 0.984000
[[611   7]
 [  3   4]]
             precision    recall  f1-score   support

          0       1.00      0.99      0.99       618
          1       0.36      0.57      0.44         7

avg / total       0.99      0.98      0.99       625

Overall Accuracy: 0.98
Number of support vectors for each class:  [93 27]


Number of support vectors for each class:  [1265   27]


### adjust with sample weights

In [22]:
def func(x):
    x = np.where(x==0, 1, 4)
    return x 

sw = func(y_train)

In [35]:
#fit with weight
svm = SVC(C=100.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=seed, shrinking=True,
    tol=0.001, verbose=False)



svm.fit(X_train, y_train, sample_weight = sw)
svmpred = svm.predict(X_test)

print("Training set score: %f" % svm.score(X_train, y_train))
print("Test set score: %f" % svm.score(X_test, y_test))

# Get the confusion matrix for your classifier using metrics.confusion_matrix
mat = metrics.confusion_matrix(y_test, svmpred)    
print(mat)


print(metrics.classification_report(y_test, svmpred))
print("Overall Accuracy:", round(metrics.accuracy_score(y_test, svmpred),2))


##get support vector details
#print ("Support Vectors: ", svm.support_vectors_)

# get number of support vectors for each class 
print("Number of support vectors for each class: ", svm.n_support_)

#Indices of support vectors
#print("Support vector indices: ", svm.support_)

Training set score: 0.959423
Test set score: 0.948800
[[588  30]
 [  2   5]]
             precision    recall  f1-score   support

          0       1.00      0.95      0.97       618
          1       0.14      0.71      0.24         7

avg / total       0.99      0.95      0.97       625

Overall Accuracy: 0.95
Number of support vectors for each class:  [623  23]


#### SGD Classifier

#### Perceptron

In [28]:

p_clf = Perceptron(penalty = 'l2',alpha = 0.001,fit_intercept = True, n_iter = 25, shuffle = True, verbose = 0, 
                   eta0 = 0.1, n_jobs = 1, random_state = seed, class_weight = 'balanced', warm_start = False)


p_clf.fit(X_train, y_train, sample_weight = sw)



p_pred = p_clf.predict(X_test)


print("Training set score: %f" % p_clf.score(X_train, y_train))
print("Test set score: %f" % p_clf.score(X_test, y_test))

mat = metrics.confusion_matrix(y_test, p_pred)    
print(mat)

sw_test = func(y_test)
print(metrics.classification_report(y_test, p_pred))
print("Overall Accuracy:", round(metrics.accuracy_score(y_test, p_pred, sample_weight = sw_test),2))

Training set score: 0.961025
Test set score: 0.939200
[[580  38]
 [  0   7]]
             precision    recall  f1-score   support

          0       1.00      0.94      0.97       618
          1       0.16      1.00      0.27         7

avg / total       0.99      0.94      0.96       625

Overall Accuracy: 0.94


##### use the perceptron for cross validation

In [30]:
#10-fold CV
cv_p_clf = Perceptron(penalty = 'l2',alpha = 0.001,fit_intercept = True, n_iter = 25, shuffle = True, verbose = 0, 
                   eta0 = 0.1, n_jobs = 1, random_state = 1, class_weight = 'balanced', warm_start = False)


scores = cross_val_score(cv_p_clf, X_train, y_train, cv = 2, scoring = 'recall', fit_params = {'sample_weight':sw})
prec_scores = cross_val_score(cv_p_clf, X_train, y_train, cv = 2, scoring = 'precision', fit_params = {'sample_weight':sw})

print ("Recalls: ", scores)
print ("Average Recall: ", scores.mean())

print ("Precisions:", prec_scores)
print ("Average Precision:", prec_scores.mean())

Recalls:  [ 0.625  0.625]
Average Recall:  0.625
Precisions: [ 0.12820513  0.12820513]
Average Precision: 0.128205128205


In [33]:
X_CV = scipy_matrix
print ("Done transforming")

Done transforming


In [38]:
#np.random.seed(55)
## use stratified 5X2-fold CV
# first merge and preprocess data
print ("starting")


p_precision = []
p_recall = []
p_f1 = []
print ("Fitting the model...")

cv_p_clf = Perceptron(penalty = 'l2',alpha = 0.001,fit_intercept = True, n_iter = 10, shuffle = True, verbose = 0, 
                   eta0 = 0.1, n_jobs = 1, random_state = seed, class_weight = 'balanced', warm_start = False)

for i in range(5):
    skf = StratifiedKFold(y_CV, n_folds=2, shuffle = True, random_state = seed)
    for train_index, test_index in skf:
        
        CV_train, CV_test = X_CV[train_index], X_CV[test_index]
        y_CVtrain, y_CVtest = y_CV[train_index], y_CV[test_index]
    
        sw = func(y_CVtrain)
        wt = func(y_CVtest)
        
        cv_p_clf.fit(CV_train, y_CVtrain, sample_weight = sw)
        cv_p_pred = cv_p_clf.predict(CV_test)
        
        p_prec = precision_score(y_CVtest, cv_p_pred, labels=None, pos_label=1, average='binary', sample_weight=wt)
        p_rec = recall_score(y_CVtest, cv_p_pred, labels=None, pos_label=1, average='binary', sample_weight=wt)
        p_f = f1_score(y_CVtest, cv_p_pred, labels=None, pos_label=1, average='binary', sample_weight=wt)
        
        
        p_precision.append(p_prec)
        p_recall.append(p_rec)
        p_f1.append(p_f)
        
        
print ("Perceptron Cross validation results: ")
print ("Precision: %.2f "%(np.mean(p_precision)))
print ("   Recall: %.2f " %(np.mean(p_recall)))
print ("       F1: %.2f " %(np.mean(p_f1)))

print ("done")

starting
Fitting the model...
Perceptron Cross validation results: 
Precision: 0.02 
   Recall: 0.08 
       F1: 0.03 
done
